In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mosplot.plot import load_lookup_table, Mosfet, Expression
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interactive_output, HBox, VBox
import matplotlib.ticker as ticker 

In [2]:
lookup_table_nmos = load_lookup_table('../../../module_0_foundations/sg13_nmos_lv_LUT.npz')
lookup_table_pmos = load_lookup_table('../../../module_0_foundations/sg13_pmos_lv_LUT.npz')

In [3]:
print(lookup_table_nmos.keys())

dict_keys(['sg13_lv_nmos ', 'description', 'simulator', 'parameter_names', 'device_parameters'])


In [4]:
nmos = Mosfet(lookup_table=lookup_table_nmos, mos="sg13_lv_nmos ", vbs=0.0, vds=0.6)
pmos = Mosfet(lookup_table=lookup_table_pmos, mos="sg13_lv_pmos", vbs=0.0, vds=-0.6, vgs=(-1.2, -0.15))

rows_0, cols_0 = np.shape(nmos.extracted_table['gm']) # just for getting the shape of the data
rows_1, cols_1 = np.shape(pmos.extracted_table['gm']) # just for getting the shape of the data
reshaped_lengths_nmos = np.tile(nmos.length[:, np.newaxis], (1, cols_0))
reshaped_lengths_pmos = np.tile(pmos.length[:, np.newaxis], (1, cols_1))

In [5]:
def plot_data_vs_data(x_values, y_values, z_values, length, x_axis_name, y_axis_name='y', y_multiplier=1, log=False):
    x_values_flat = np.array(x_values).flatten()
    y_values_flat = np.array(y_values, dtype=np.float64).flatten()
    z_values_flat = np.array(z_values, dtype=np.float64).flatten()
    length_flat = np.array(length).flatten()

    # Ensure all inputs have the same length
    if not (len(x_values_flat) == len(y_values_flat) == len(z_values_flat) == len(length_flat)):
        raise ValueError("All input arrays (x_values, y_values, z_values, length) must have the same number of elements.")

    unique_lengths = np.unique(length_flat)
    unique_lengths_in_micro = unique_lengths * 1e6

    def update_plot(selected_length, x_value=None, y_value=None):
        plt.figure(figsize=(12, 8))  # Make the figure wider (adjust as needed)

        if selected_length == "Show All":
            mask = np.ones_like(length_flat, dtype=bool)
        else:
            selected_length_in_micro = float(selected_length.replace(' μm', ''))
            tolerance = 0.01  # Tighten the tolerance to avoid unwanted data points
            mask = np.abs(length_flat * 1e6 - selected_length_in_micro) < tolerance

        # Apply the mask to the data
        x_values_for_length = x_values_flat[mask]
        y_values_for_length = y_values_flat[mask] * y_multiplier
        z_values_for_length = z_values_flat[mask]
        length_for_length = length_flat[mask] * 1e6

        if selected_length == "Show All":
            for length_value in np.unique(length_for_length):
                mask_all = (length_for_length == length_value)
                plt.plot(x_values_for_length[mask_all], y_values_for_length[mask_all])

            min_length = np.min(unique_lengths_in_micro)
            max_length = np.max(unique_lengths_in_micro)
            plt.title(f'{y_axis_name} vs {x_axis_name} (Length from {min_length:.2f} μm to {max_length:.2f} μm)')

        else:
            plt.plot(x_values_for_length, y_values_for_length)
            plt.title(f'{y_axis_name} vs {x_axis_name} for {selected_length}')

        plt.xlabel(f'{x_axis_name}')
        plt.ylabel(f'{y_axis_name}')

        if log:
            plt.yscale('log')
            plt.gca().yaxis.set_major_locator(ticker.LogLocator(base=10, subs=[], numticks=10))
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'$10^{int(np.log10(x))}$'))
            plt.ylabel(f'{y_axis_name} (Log Base 10)')

        if y_value is not None and x_value_widget.disabled:
            closest_index = np.abs(y_values_for_length - y_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='blue', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {x_axis_name} value for {y_axis_name} = {closest_y:.2f} is: {closest_x:.2f}")
        elif x_value is not None and y_value_widget.disabled:
            closest_index = np.abs(x_values_for_length - x_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='red', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {y_axis_name} value for {x_axis_name} = {closest_x:.2f} is: {closest_y:.2f}")

        plt.grid(True)
        plt.legend()
        plt.show()

    dropdown_options = ["Show All"] + [f'{length:.2f} μm' for length in unique_lengths_in_micro]
    length_widget = widgets.Dropdown(
        options=dropdown_options,
        value=dropdown_options[0],
        description='Length:',
        layout=widgets.Layout(width='500px')  # Make the dropdown wider
    )

    x_value_widget = widgets.FloatText(
        value=np.mean(x_values_flat),
        description=f"{x_axis_name}:",
        disabled=False,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    y_value_widget = widgets.FloatText(
        value=None,
        description=f"{y_axis_name}:",
        disabled=True,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    z_value_widget = widgets.FloatText(
        value=None,
        description=f" Vgs:",
        disabled=True,
        layout=widgets.Layout(width='300px', margin='0 40px 0 0'),  # Push input boxes more to the right
        description_width='150px'  # Smaller description width
    )

    select_x_or_y_widget = widgets.Checkbox(
        value=True,
        description=f"{x_axis_name} (uncheck for {y_axis_name})",
        layout=widgets.Layout(width='300px')  # Make the checkbox wider
    )

    def toggle_x_or_y(change):
        if change['new']:
            x_value_widget.disabled = False
            y_value_widget.disabled = True
        else:
            x_value_widget.disabled = True
            y_value_widget.disabled = False

    select_x_or_y_widget.observe(toggle_x_or_y, names='value')

    output = interactive_output(update_plot, {
        'selected_length': length_widget,
        'x_value': x_value_widget,
        'y_value': y_value_widget
    })

    display(VBox([length_widget, select_x_or_y_widget, HBox([x_value_widget, y_value_widget]), z_value_widget, output]))


def display_resistance(ro_value):
    """Determine the resistance value and its unit."""
    if ro_value < 1e3:
        return ro_value, "Ω"
    elif ro_value < 1e6:
        return ro_value / 1e3, "kΩ"
    elif ro_value < 1e9:
        return ro_value / 1e6, "MΩ"
    else:
        return ro_value / 1e9, "GΩ"

def display_current(Id_value):
    """Determine the current value and its unit."""
    if Id_value < 1e-6:
        return Id_value * 1e9, "nA"  # Convert to nA
    elif Id_value < 1e-3:
        return Id_value * 1e6, "μA"  # Convert to μA
    else:
        return Id_value * 1e3, "mA"   # Convert to mA
    
def dB_to_linear(av_db):
    return 10 ** (av_db / 20)


def determine_inversion_region(gm_id_value, device_type):
    """Determine the inversion region based on gm/id value for NMOS or PMOS."""
    if device_type == 'nmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    elif device_type == 'pmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    else:
        raise ValueError("Invalid device type. Use 'nmos' or 'pmos'.")
    


In [34]:
Av_target = 37 # db
Av_linear = dB_to_linear(Av_target)
ID = 3e-6
Cl = 700e-15
fp1 = 50e3
Ro = 1/(2*np.pi * fp1 * Cl)
gm_input = Av_linear / Ro
gmid_input_transistor = gm_input / ID

gmid_input_transistor

5.1895136425091675



$$A_v = gm_{1,2} \cdot R_{p} \vert \vert R_{n}$$

In [35]:
gmro_input = 2 * Ro * gm_input
gmro_input

141.58915687682762

In [8]:
pmos = Mosfet(lookup_table=lookup_table_pmos, mos="sg13_lv_pmos", vbs=0, vds=-0.6, vgs=(-1.2, -0.2))
rows_0, cols_0 = np.shape(pmos.extracted_table['gm'])
reshaped_lengths_pmos = np.tile(pmos.length[:, np.newaxis], (1, cols_0))

width_values_pmos = pmos.width
id_values_pmos = pmos.extracted_table['id']
gm_values_pmos = pmos.extracted_table['gm']
gds_values_pmos = pmos.extracted_table['gds']
vgs_values_pmos = pmos.extracted_table['vgs']

plot_data_vs_data(gm_values_pmos/id_values_pmos, gm_values_pmos/gds_values_pmos, vgs_values_pmos, reshaped_lengths_pmos, 'gm/id', 'gds')




In [36]:
L_input_pmos = 5.46e-6
gmro_input_pmos = 212.47

In [40]:
ro_nmos = Ro
gmid_nmos = 8
gm_nmos = gmid_nmos * ID
gmro_nmos = gm_nmos * ro_nmos

In [11]:
nmos = Mosfet(lookup_table=lookup_table_nmos, mos="sg13_lv_nmos ", vbs=0, vds=0.6, vgs=(0.2, 1.0))
rows_0, cols_0 = np.shape(nmos.extracted_table['gm'])
reshaped_lengths_nmos = np.tile(nmos.length[:, np.newaxis], (1, cols_0))

width_values_nmos = nmos.width
id_values_nmos = nmos.extracted_table['id']
gm_values_nmos = nmos.extracted_table['gm']
gds_values_nmos = nmos.extracted_table['gds']
vgs_values_nmos = nmos.extracted_table['vgs']

plot_data_vs_data(gm_values_nmos/id_values_nmos, gm_values_nmos/gds_values_nmos, vgs_values_nmos, reshaped_lengths_nmos, 'gm/id', 'gm/gds')

In [12]:
L_nmos = 9.98e-6
gmro_nmos = 64.7

In [13]:
plot_data_vs_data(gm_values_nmos/id_values_nmos,id_values_nmos/width_values_nmos, vgs_values_nmos, reshaped_lengths_nmos, 'gm/id', 'nmos id/W', log=True)

In [41]:
idW_nmos = 0.97
W_nmos = ID/ idW_nmos
W_nmos

3.0927835051546395e-06

In [15]:
plot_data_vs_data(gm_values_pmos/id_values_pmos,id_values_pmos/width_values_pmos, vgs_values_pmos, reshaped_lengths_pmos, 'gm/id', 'nmos id/W', log=True)

In [42]:
idW_pmos = 1.09
W_pmos = ID / idW_pmos

In [43]:
print(f"PMOS Width (W_pmos):       {W_pmos * 1e6:.2f} µm")
print(f"PMOS Length (L_input_pmos): {L_input_pmos * 1e6:.2f} µm")
print(f"NMOS Width (W_nmos):        {W_nmos * 1e6:.2f} µm")
print(f"NMOS Length (L_nmos):       {L_nmos * 1e6:.2f} µm")
print(f"Tail Current:       {(ID * 2)*1e6} µA")


PMOS Width (W_pmos):       2.75 µm
PMOS Length (L_input_pmos): 5.46 µm
NMOS Width (W_nmos):        3.09 µm
NMOS Length (L_nmos):       6.24 µm
Tail Current:       6.0 µA


In [44]:
# total gain
gm_nmos = gmid_nmos * ID
ro_nmos = gmro_nmos / gm_nmos
ro_pmos = gmro_input_pmos / gm_input 

RO_actual = (ro_nmos * ro_pmos)/(ro_nmos + ro_pmos)
Gain_actual = RO_actual * gm_input
Gain_actual

53.10132372968084

In [45]:
20*np.log10(53.1)

np.float64(34.50189042162938)

In [27]:
#current_mirror_pmos
CMpmos_gmid = 18
Itail = 2 * ID

In [23]:
pmos = Mosfet(lookup_table=lookup_table_pmos, mos="sg13_lv_pmos", vbs=0, vds=-0.2, vgs=(-1.2, -0.2))
rows_0, cols_0 = np.shape(pmos.extracted_table['gm'])
reshaped_lengths_pmos = np.tile(pmos.length[:, np.newaxis], (1, cols_0))

width_values_pmos = pmos.width
id_values_pmos = pmos.extracted_table['id']
gm_values_pmos = pmos.extracted_table['gm']
gds_values_pmos = pmos.extracted_table['gds']
vgs_values_pmos = pmos.extracted_table['vgs']

plot_data_vs_data(gm_values_pmos/id_values_pmos, gm_values_pmos/gds_values_pmos, vgs_values_pmos, reshaped_lengths_pmos, 'gm/id', 'gds')




In [24]:
plot_data_vs_data(gm_values_pmos/id_values_pmos,id_values_pmos/width_values_pmos, vgs_values_pmos, reshaped_lengths_pmos, 'gm/id', 'nmos id/W', log=True)

In [29]:
CM_pmos_L = 9.98e-6
id_over_w_CM_pmos = 0.03
W = Itail / id_over_w_CM_pmos
W

6.666666666666667e-05